In [1]:
import pandas as pd

In [10]:
#그래프에 한글 표현하기
import  matplotlib.pyplot as plt
import  matplotlib.font_manager as fm
font_name = fm.FontProperties(fname = "malgun.ttf").get_name()
plt.rc("font", family = font_name)

#마이너스 표시하기
import  matplotlib as mlp
mlp.rcParams["axes.unicode_minus"]  = False

In [11]:
import numpy as np
import pandas  as pd
##회귀분석
import statsmodels.api as sm
# 정규화
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import  RFE
from lightgbm import LGBMClassifier,  LGBMRegressor
from lightgbm import plot_importance, plot_metric, plot_tree
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR, SVC
import multiprocessing 
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import  LinearRegression
from scipy import stats
#사용법이 파이선과 다른 기존xgb를  파이선에 맞게 만들어 놓은 것
from  xgboost import XGBClassifier, XGBRegressor

#변수영향도 그래프, 트리그래프 더 쉽게 그리게 해줌
from xgboost import plot_importance, plot_tree

In [8]:
df = pd.read_csv("*회귀분석_위한_전처리완성본/절도폭력안전도.csv")
df.head()

,jur_stn,연도,crm_wthr_눈,crm_wthr_만월,crm_wthr_맑음,crm_wthr_미상,crm_wthr_바람,crm_wthr_비,crm_wthr_안개,crm_wthr_암흑,...,"vic_age_2,30대","vic_age_4,50대",vic_age_60세초과,vic_age_미상,crm_tm_미상,crm_tm_새벽,crm_tm_오전,crm_tm_오후,crm_tm_저녁,score_절폭
0,경기수원남부경찰서,2017,7.0,2.0,4072.0,1466.0,1.0,61.0,0.0,219.0,...,2784.0,1817.0,318.0,344.0,691.0,1444.0,925.0,1122.0,1684.0,78.30
1,경기수원서부경찰서,2017,3.0,0.0,1869.0,1462.0,0.0,50.0,0.0,98.0,...,1418.0,1196.0,297.0,223.0,340.0,749.0,512.0,749.0,1207.0,70.10
2,경기수원중부경찰서,2017,5.0,12.0,2849.0,1273.0,0.0,61.0,1.0,38.0,...,1284.0,1738.0,484.0,373.0,362.0,769.0,623.0,938.0,1582.0,73.40
3,경남마산동부경찰서,2017,0.0,24.0,1312.0,563.0,1.0,12.0,0.0,39.0,...,593.0,798.0,243.0,170.0,226.0,426.0,294.0,348.0,660.0,73.15
4,경남마산중부경찰서,2017,0.0,20.0,955.0,257.0,0.0,11.0,1.0,111.0,...,368.0,572.0,218.0,130.0,127.0,313.0,222.0,272.0,443.0,77.85


In [7]:
df_검거율 = pd.read_csv("*전처리완성본/0_관할서별_연도별_검거율현황", index_col=0)
df_검거율.head()

,jur_stn,date,절도폭력_검거율,강도살인_검거율,교통사고_검거율,법질서_검거율,전반적_검거율
0,경기수원남부경찰서,2017,0.737,1.077,0.868,0.918,0.849
1,경기수원남부경찰서,2018,0.730,1.154,0.995,0.905,0.875
2,경기수원남부경찰서,2019,0.850,1.765,1.011,1.086,0.992
3,경기수원서부경찰서,2017,0.766,1.500,0.968,0.878,0.860
4,경기수원서부경찰서,2018,0.755,1.000,1.030,0.812,0.839


In [9]:

df_발생수 = pd.read_csv("*전처리완성본/6_범죄발생원표_전처리완성본", index_col=0)
df_발생수.head()

,jur_stn,crm_date,절도폭력_발생수,강도살인_발생수,교통사고_발생수,법질서_발생수,전반적_발생수
0,경기수원남부경찰서,2017,5866,13,5507,8098,19756
1,경기수원남부경찰서,2018,5670,13,5038,6840,17846
2,경기수원남부경찰서,2019,5187,17,4224,6488,16148
3,경기수원서부경찰서,2017,3557,10,1991,5524,11254
4,경기수원서부경찰서,2018,3498,9,2041,4109,9799


In [10]:
df_검거수 = pd.read_csv("*전처리완성본/7_범죄검거원표_전처리완성본", index_col=0)
df_검거수.head()

,jur_stn,date,절도폭력_검거수,강도살인_검거수,교통사고_검거수,법질서_검거수,전반적_검거수
0,경기수원남부경찰서,2017,4324,14,4779,7432,16775
1,경기수원남부경찰서,2018,4141,15,5014,6193,15619
2,경기수원남부경찰서,2019,4407,30,4272,7047,16021
3,경기수원서부경찰서,2017,2726,15,1928,4852,9682
4,경기수원서부경찰서,2018,2641,9,2103,3336,8220


In [11]:
df_비상벨수 = pd.read_csv("*전처리완성본/22_관할서별_비상벨개수.csv", index_col=0)
df_비상벨수.head()

,관할서,bell_cnt,date
0,경기수원남부경찰서,5.0,2017
1,경기수원서부경찰서,1.0,2017
2,경기수원중부경찰서,8.0,2017
3,경남마산동부경찰서,11.0,2017
4,경남마산중부경찰서,12.0,2017


In [37]:
df_mid = pd.concat([df,df_검거율["절도폭력_검거율"],df_발생수["절도폭력_발생수"],df_검거수["절도폭력_검거수"],df_비상벨수["bell_cnt"]], axis = 1)
df_mid.head()

,jur_stn,연도,crm_wthr_눈,crm_wthr_만월,crm_wthr_맑음,crm_wthr_미상,crm_wthr_바람,crm_wthr_비,crm_wthr_안개,crm_wthr_암흑,...,crm_tm_미상,crm_tm_새벽,crm_tm_오전,crm_tm_오후,crm_tm_저녁,score_절폭,절도폭력_검거율,절도폭력_발생수,절도폭력_검거수,bell_cnt
0,경기수원남부경찰서,2017,7.0,2.0,4072.0,1466.0,1.0,61.0,0.0,219.0,...,691.0,1444.0,925.0,1122.0,1684.0,78.30,0.737,5866,4324,5.0
1,경기수원서부경찰서,2017,3.0,0.0,1869.0,1462.0,0.0,50.0,0.0,98.0,...,340.0,749.0,512.0,749.0,1207.0,70.10,0.730,5670,4141,1.0
2,경기수원중부경찰서,2017,5.0,12.0,2849.0,1273.0,0.0,61.0,1.0,38.0,...,362.0,769.0,623.0,938.0,1582.0,73.40,0.850,5187,4407,8.0
3,경남마산동부경찰서,2017,0.0,24.0,1312.0,563.0,1.0,12.0,0.0,39.0,...,226.0,426.0,294.0,348.0,660.0,73.15,0.766,3557,2726,11.0
4,경남마산중부경찰서,2017,0.0,20.0,955.0,257.0,0.0,11.0,1.0,111.0,...,127.0,313.0,222.0,272.0,443.0,77.85,0.755,3498,2641,12.0


In [24]:
df_cctv = pd.read_csv("*전처리완성본/10_관할서별_cctv개수", index_col = 0)
df_cctv

,관할서,cctv_cnt
0,경기수원남부경찰서,4719
1,경기수원서부경찰서,2919
2,경기수원중부경찰서,4447
3,경남마산동부경찰서,1218
4,경남마산중부경찰서,913
5,경남진주경찰서,2709
6,경남진해경찰서,1142
7,경남창원서부경찰서,1050
8,경남창원중부경찰서,1429
9,서울강남경찰서,3095


In [25]:
df_보안등 = pd.read_csv("*전처리완성본/9_관할서별_보안등개수", index_col = 0)
df_보안등.head()

,관할서,securitylight_cnt
0,경기수원남부경찰서,3671
1,경기수원서부경찰서,3961
2,경기수원중부경찰서,6736
3,경남마산동부경찰서,5647
4,경남마산중부경찰서,6843


In [26]:
df_배치인원 = pd.read_csv("*전처리완성본/12_관할서별_배치인원현황", index_col = 0)
df_배치인원.head()

,관할서,pol_pop
0,경기수원남부경찰서,395
1,경기수원서부경찰서,258
2,경기수원중부경찰서,333
3,경남마산동부경찰서,185
4,경남마산중부경찰서,142


In [27]:
df_유흥업소 = pd.read_csv("*전처리완성본/21_관할서별_유흥업소 개수", index_col = 0)
df_유흥업소.head()

,관할서,ent_num
0,경기수원남부경찰서,223
1,경기수원서부경찰서,79
2,경기수원중부경찰서,157
3,경남마산동부경찰서,337
4,경남마산중부경찰서,487


In [28]:
df_공원 = pd.read_csv("*전처리완성본/20_관할서별_공원개수", index_col = 0)
df_공원.head()

,관할서,park_nm
0,경기수원남부경찰서,148
1,경기수원서부경찰서,102
2,경기수원중부경찰서,96
3,경남마산동부경찰서,65
4,경남마산중부경찰서,55


In [38]:
df_mid  = df_mid.rename(columns = {"jur_stn":"관할서"})
df_mid.head()

,관할서,연도,crm_wthr_눈,crm_wthr_만월,crm_wthr_맑음,crm_wthr_미상,crm_wthr_바람,crm_wthr_비,crm_wthr_안개,crm_wthr_암흑,...,crm_tm_미상,crm_tm_새벽,crm_tm_오전,crm_tm_오후,crm_tm_저녁,score_절폭,절도폭력_검거율,절도폭력_발생수,절도폭력_검거수,bell_cnt
0,경기수원남부경찰서,2017,7.0,2.0,4072.0,1466.0,1.0,61.0,0.0,219.0,...,691.0,1444.0,925.0,1122.0,1684.0,78.30,0.737,5866,4324,5.0
1,경기수원서부경찰서,2017,3.0,0.0,1869.0,1462.0,0.0,50.0,0.0,98.0,...,340.0,749.0,512.0,749.0,1207.0,70.10,0.730,5670,4141,1.0
2,경기수원중부경찰서,2017,5.0,12.0,2849.0,1273.0,0.0,61.0,1.0,38.0,...,362.0,769.0,623.0,938.0,1582.0,73.40,0.850,5187,4407,8.0
3,경남마산동부경찰서,2017,0.0,24.0,1312.0,563.0,1.0,12.0,0.0,39.0,...,226.0,426.0,294.0,348.0,660.0,73.15,0.766,3557,2726,11.0
4,경남마산중부경찰서,2017,0.0,20.0,955.0,257.0,0.0,11.0,1.0,111.0,...,127.0,313.0,222.0,272.0,443.0,77.85,0.755,3498,2641,12.0


In [39]:
df_fin  = pd.merge(df_mid, df_cctv,how = "left", on=["관할서"])
df_fin  = pd.merge(df_fin, df_보안등,how = "left", on=["관할서"])
df_fin  = pd.merge(df_fin, df_공원,how = "left", on=["관할서"])
df_fin  = pd.merge(df_fin, df_배치인원,how = "left", on=["관할서"])
df_fin  = pd.merge(df_fin, df_유흥업소,how = "left", on=["관할서"])


In [40]:
df_fin.isnull().sum()

관할서                  0
연도                   0
crm_wthr_눈           0
crm_wthr_만월          0
crm_wthr_맑음          0
crm_wthr_미상          0
crm_wthr_바람          0
crm_wthr_비           0
crm_wthr_안개          0
crm_wthr_암흑          0
crm_wthr_폭설          0
crm_wthr_폭풍우         0
crm_wthr_흐림          0
crm_clue_고발          0
crm_clue_고소          0
crm_clue_기타          0
crm_clue_변사체         0
crm_clue_불심검문        0
crm_clue_여죄          0
crm_clue_자수          0
crm_clue_진정          0
crm_clue_타인신고        0
crm_clue_탄원          0
crm_clue_탐문정보        0
crm_clue_투서          0
crm_clue_피해자신고       0
crm_clue_피해품발견       0
crm_clue_현행범         0
vic_sx_1             0
vic_sx_2             0
vic_sx_3             0
vic_age_0~12세        0
vic_age_10대          0
vic_age_2,30대        0
vic_age_4,50대        0
vic_age_60세초과        0
vic_age_미상           0
crm_tm_미상            0
crm_tm_새벽            0
crm_tm_오전            0
crm_tm_오후            0
crm_tm_저녁            0
score_절폭             0
절도폭력_검거율   

In [42]:
df_fin.to_csv("절도폭력완성본")

In [2]:
df = pd.read_csv("절도폭력완성본")

In [3]:
df.columns

Index(['관할서', '연도', 'crm_wthr_눈', 'crm_wthr_만월', 'crm_wthr_맑음', 'crm_wthr_미상',
       'crm_wthr_바람', 'crm_wthr_비', 'crm_wthr_안개', 'crm_wthr_암흑',
       'crm_wthr_폭설', 'crm_wthr_폭풍우', 'crm_wthr_흐림', 'crm_clue_고발',
       'crm_clue_고소', 'crm_clue_기타', 'crm_clue_변사체', 'crm_clue_불심검문',
       'crm_clue_여죄', 'crm_clue_자수', 'crm_clue_진정', 'crm_clue_타인신고',
       'crm_clue_탄원', 'crm_clue_탐문정보', 'crm_clue_투서', 'crm_clue_피해자신고',
       'crm_clue_피해품발견', 'crm_clue_현행범', 'vic_sx_1', 'vic_sx_2', 'vic_sx_3',
       'vic_age_0~12세', 'vic_age_10대', 'vic_age_2,30대', 'vic_age_4,50대',
       'vic_age_60세초과', 'vic_age_미상', 'crm_tm_미상', 'crm_tm_새벽', 'crm_tm_오전',
       'crm_tm_오후', 'crm_tm_저녁', 'score_절폭', '절도폭력_검거율', '절도폭력_발생수',
       '절도폭력_검거수', 'bell_cnt', 'cctv_cnt', 'securitylight_cnt', 'park_nm',
       'pol_pop', 'ent_num'],
      dtype='object')

In [4]:
df = df[['관할서', '연도', 'crm_wthr_눈', 'crm_wthr_만월', 'crm_wthr_맑음', 'crm_wthr_미상',
       'crm_wthr_바람', 'crm_wthr_비', 'crm_wthr_안개', 'crm_wthr_암흑',
       'crm_wthr_폭설', 'crm_wthr_폭풍우', 'crm_wthr_흐림', 'crm_clue_고발',
       'crm_clue_고소', 'crm_clue_기타', 'crm_clue_변사체', 'crm_clue_불심검문',
       'crm_clue_여죄', 'crm_clue_자수', 'crm_clue_진정', 'crm_clue_타인신고',
       'crm_clue_탄원', 'crm_clue_탐문정보', 'crm_clue_투서', 'crm_clue_피해자신고',
       'crm_clue_피해품발견', 'crm_clue_현행범', 'vic_sx_1', 'vic_sx_2', 'vic_sx_3',
       'vic_age_0~12세', 'vic_age_10대', 'vic_age_2,30대', 'vic_age_4,50대',
       'vic_age_60세초과', 'vic_age_미상', 'crm_tm_미상', 'crm_tm_새벽', 'crm_tm_오전',
       'crm_tm_오후', 'crm_tm_저녁', '절도폭력_검거율', '절도폭력_발생수',
       '절도폭력_검거수', 'bell_cnt', 'cctv_cnt', 'securitylight_cnt', 'park_nm',
       'pol_pop', 'ent_num', 'score_절폭']]

In [35]:
df = pd.read_csv("절도폭력완성본")
df.head()

,관할서,연도,crm_wthr_눈,crm_wthr_만월,crm_wthr_맑음,crm_wthr_미상,crm_wthr_바람,crm_wthr_비,crm_wthr_안개,crm_wthr_암흑,...,절도폭력_검거율,절도폭력_발생수,절도폭력_검거수,bell_cnt,cctv_cnt,securitylight_cnt,park_nm,pol_pop,ent_num,score_절폭
0,경기수원남부경찰서,2017,7.0,2.0,4072.0,1466.0,1.0,61.0,0.0,219.0,...,0.737,5866,4324,5.0,4719,3671,148,395,223,78.30
1,경기수원서부경찰서,2017,3.0,0.0,1869.0,1462.0,0.0,50.0,0.0,98.0,...,0.730,5670,4141,1.0,2919,3961,102,258,79,70.10
2,경기수원중부경찰서,2017,5.0,12.0,2849.0,1273.0,0.0,61.0,1.0,38.0,...,0.850,5187,4407,8.0,4447,6736,96,333,157,73.40
3,경남마산동부경찰서,2017,0.0,24.0,1312.0,563.0,1.0,12.0,0.0,39.0,...,0.766,3557,2726,11.0,1218,5647,65,185,337,73.15
4,경남마산중부경찰서,2017,0.0,20.0,955.0,257.0,0.0,11.0,1.0,111.0,...,0.755,3498,2641,12.0,913,6843,55,142,487,77.85


In [40]:
df_sc = df.iloc[:,2:-1]
df_sc.columns

Index(['crm_wthr_눈', 'crm_wthr_만월', 'crm_wthr_맑음', 'crm_wthr_미상',
       'crm_wthr_바람', 'crm_wthr_비', 'crm_wthr_안개', 'crm_wthr_암흑',
       'crm_wthr_폭설', 'crm_wthr_폭풍우', 'crm_wthr_흐림', 'crm_clue_고발',
       'crm_clue_고소', 'crm_clue_기타', 'crm_clue_변사체', 'crm_clue_불심검문',
       'crm_clue_여죄', 'crm_clue_자수', 'crm_clue_진정', 'crm_clue_타인신고',
       'crm_clue_탄원', 'crm_clue_탐문정보', 'crm_clue_투서', 'crm_clue_피해자신고',
       'crm_clue_피해품발견', 'crm_clue_현행범', 'vic_sx_1', 'vic_sx_2', 'vic_sx_3',
       'vic_age_0~12세', 'vic_age_10대', 'vic_age_2,30대', 'vic_age_4,50대',
       'vic_age_60세초과', 'vic_age_미상', 'crm_tm_미상', 'crm_tm_새벽', 'crm_tm_오전',
       'crm_tm_오후', 'crm_tm_저녁', '절도폭력_검거율', '절도폭력_발생수', '절도폭력_검거수',
       'bell_cnt', 'cctv_cnt', 'securitylight_cnt', 'park_nm', 'pol_pop',
       'ent_num'],
      dtype='object')

In [38]:
def standard_scaling(df, scale_columns):
    for col in scale_columns:
        series_mean = df[col].mean()
        series_std = df[col].std()
        df[col] = df[col].apply(lambda x: (x-series_mean)/series_std)
    return df

In [41]:
standard_scaling(df,df_sc.columns )

,관할서,연도,crm_wthr_눈,crm_wthr_만월,crm_wthr_맑음,crm_wthr_미상,crm_wthr_바람,crm_wthr_비,crm_wthr_안개,crm_wthr_암흑,...,절도폭력_검거율,절도폭력_발생수,절도폭력_검거수,bell_cnt,cctv_cnt,securitylight_cnt,park_nm,pol_pop,ent_num,score_절폭
0,경기수원남부경찰서,2017,1.441782,-0.122371,2.687666,1.029634,0.213519,1.237725,-0.254478,0.697511,...,-0.322573,2.298230,2.326356,-0.909420,1.606895,-1.113231,1.892891,0.749224,0.395463,78.30
1,경기수원서부경찰서,2017,0.137817,-0.375338,0.149152,1.021427,-0.427038,0.655444,-0.254478,-0.327475,...,-0.416087,2.144964,2.126072,-0.918843,0.310941,-0.992471,0.790701,-0.485270,-0.758101,70.10
2,경기수원중부경찰서,2017,0.789800,1.142468,1.278405,0.633625,-0.427038,1.237725,1.981292,-0.835733,...,1.187005,1.767275,2.417195,-0.902353,1.411062,0.163071,0.646937,0.190548,-0.133254,73.40
3,경남마산동부경찰서,2017,-0.840156,2.660275,-0.492678,-0.823197,0.213519,-1.356075,-0.254478,-0.827262,...,0.064841,0.492670,0.577431,-0.895286,-0.913735,-0.290401,-0.095843,-1.143066,1.308700,73.15
4,경남마산중부경찰서,2017,-0.840156,2.154339,-0.904048,-1.451067,-0.427038,-1.409010,1.981292,-0.217353,...,-0.082110,0.446534,0.484403,-0.892930,-1.133327,0.207627,-0.335449,-1.530535,2.510329,77.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,서울종암경찰서,2019,-0.514165,-0.248854,-1.195580,-0.749330,-0.427038,-0.773793,-0.254478,-0.793378,...,-1.217633,-1.057968,-1.251387,0.254327,-0.455112,-0.247095,-1.126150,-1.224164,-1.222730,75.80
119,서울중랑경찰서,2019,0.137817,-0.375338,0.358871,0.883952,-0.427038,0.602509,-0.254478,0.748337,...,0.732795,-1.239384,-1.207610,3.267347,1.426901,0.744381,-0.359410,0.848344,-0.469710,71.85
120,서울중부경찰서,2019,-0.188174,-0.375338,-0.304853,-1.373096,-0.427038,1.555333,-0.254478,0.282434,...,-1.324506,-1.258933,-1.451671,-0.122595,-0.665344,-0.587719,-1.317835,-0.683510,0.259278,77.40
121,서울혜화경찰서,2019,-0.840156,-0.375338,-0.853347,-1.434652,0.213519,-0.297381,-0.254478,-0.903500,...,-2.072615,-1.215925,-1.496543,-0.822257,-1.226204,-0.869630,-1.341796,-0.890761,-0.245406,74.60


In [44]:
df.to_csv("v2절도폭력안전도_scaling", index = None)

In [46]:
df_scale = pd.read_csv("v2절도폭력안전도_scaling")
df_scale.head()

,관할서,연도,crm_wthr_눈,crm_wthr_만월,crm_wthr_맑음,crm_wthr_미상,crm_wthr_바람,crm_wthr_비,crm_wthr_안개,crm_wthr_암흑,...,절도폭력_검거율,절도폭력_발생수,절도폭력_검거수,bell_cnt,cctv_cnt,securitylight_cnt,park_nm,pol_pop,ent_num,score_절폭
0,경기수원남부경찰서,2017,1.441782,-0.122371,2.687666,1.029634,0.213519,1.237725,-0.254478,0.697511,...,-0.322573,2.298230,2.326356,-0.909420,1.606895,-1.113231,1.892891,0.749224,0.395463,78.30
1,경기수원서부경찰서,2017,0.137817,-0.375338,0.149152,1.021427,-0.427038,0.655444,-0.254478,-0.327475,...,-0.416087,2.144964,2.126072,-0.918843,0.310941,-0.992471,0.790701,-0.485270,-0.758101,70.10
2,경기수원중부경찰서,2017,0.789800,1.142468,1.278405,0.633625,-0.427038,1.237725,1.981292,-0.835733,...,1.187005,1.767275,2.417195,-0.902353,1.411062,0.163071,0.646937,0.190548,-0.133254,73.40
3,경남마산동부경찰서,2017,-0.840156,2.660275,-0.492678,-0.823197,0.213519,-1.356075,-0.254478,-0.827262,...,0.064841,0.492670,0.577431,-0.895286,-0.913735,-0.290401,-0.095843,-1.143066,1.308700,73.15
4,경남마산중부경찰서,2017,-0.840156,2.154339,-0.904048,-1.451067,-0.427038,-1.409010,1.981292,-0.217353,...,-0.082110,0.446534,0.484403,-0.892930,-1.133327,0.207627,-0.335449,-1.530535,2.510329,77.85


In [47]:
X_train, X_test, y_train, y_test  = train_test_split(df_scale.iloc[:,2:-1], df_scale["score_절폭"],test_size = 0.2)

In [48]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_train, y_train)

0.8022731143484126

In [49]:
lr.score(X_test, y_test)

-0.16258703371021266

In [51]:
mean_absolute_error(lr.predict(X_test), y_test)

2.908963006719378

In [52]:
model = SVR(kernel="linear", gamma="auto")
model.fit(X_train, y_train)

print("학습 결과 : ", model.score(X_train, y_train))
print("테스트 결과 : ", model.score(X_test, y_test))

학습 결과 :  0.7397707757616928
테스트 결과 :  -0.2879391717392523


In [53]:
mean_absolute_error(model.predict(X_test), y_test)

2.9696481493333677

In [71]:
ridge   =  Ridge(alpha = 100)
ridge.fit(X_train, y_train)
ridge.score(X_train,y_train)

0.4997472266747838

In [72]:
mean_absolute_error(ridge.predict(X_test), y_test)

2.3368772129833273

In [80]:
lasso   =  Lasso(alpha = 0.1)
lasso.fit(X_train, y_train)
mean_absolute_error(lasso.predict(X_test), y_test)

2.312981015005871

In [7]:
df.to_csv("절도폭력완성본", index = None)

In [8]:
df

,관할서,연도,crm_wthr_눈,crm_wthr_만월,crm_wthr_맑음,crm_wthr_미상,crm_wthr_바람,crm_wthr_비,crm_wthr_안개,crm_wthr_암흑,...,절도폭력_검거율,절도폭력_발생수,절도폭력_검거수,bell_cnt,cctv_cnt,securitylight_cnt,park_nm,pol_pop,ent_num,score_절폭
0,경기수원남부경찰서,2017,7.0,2.0,4072.0,1466.0,1.0,61.0,0.0,219.0,...,0.737,5866,4324,5.0,4719,3671,148,395,223,78.30
1,경기수원서부경찰서,2017,3.0,0.0,1869.0,1462.0,0.0,50.0,0.0,98.0,...,0.730,5670,4141,1.0,2919,3961,102,258,79,70.10
2,경기수원중부경찰서,2017,5.0,12.0,2849.0,1273.0,0.0,61.0,1.0,38.0,...,0.850,5187,4407,8.0,4447,6736,96,333,157,73.40
3,경남마산동부경찰서,2017,0.0,24.0,1312.0,563.0,1.0,12.0,0.0,39.0,...,0.766,3557,2726,11.0,1218,5647,65,185,337,73.15
4,경남마산중부경찰서,2017,0.0,20.0,955.0,257.0,0.0,11.0,1.0,111.0,...,0.755,3498,2641,12.0,913,6843,55,142,487,77.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,서울종암경찰서,2019,1.0,1.0,702.0,599.0,0.0,23.0,0.0,43.0,...,0.670,1574,1055,499.0,1855,5751,22,176,21,75.80
119,서울중랑경찰서,2019,3.0,0.0,2051.0,1395.0,0.0,49.0,0.0,225.0,...,0.816,1342,1095,1778.0,4469,8132,54,406,115,71.85
120,서울중부경찰서,2019,2.0,0.0,1475.0,295.0,0.0,67.0,0.0,170.0,...,0.662,1317,872,339.0,1563,4933,14,236,206,77.40
121,서울혜화경찰서,2019,0.0,0.0,999.0,265.0,1.0,32.0,0.0,30.0,...,0.606,1372,831,42.0,784,4256,13,213,143,74.60


In [14]:
df_ma = df[["crm_wthr_폭설","crm_clue_고소","crm_clue_타인신고","crm_clue_현행범","vic_age_60세초과","bell_cnt","pol_pop","ent_num","score_절폭"]]
df_ma.head()

,crm_wthr_폭설,crm_clue_고소,crm_clue_타인신고,crm_clue_현행범,vic_age_60세초과,bell_cnt,pol_pop,ent_num,score_절폭
0,0.0,148.0,148.0,774.0,318.0,5.0,395,223,78.30
1,0.0,91.0,155.0,409.0,297.0,1.0,258,79,70.10
2,0.0,143.0,77.0,850.0,484.0,8.0,333,157,73.40
3,0.0,111.0,75.0,170.0,243.0,11.0,185,337,73.15
4,0.0,73.0,47.0,72.0,218.0,12.0,142,487,77.85


In [18]:
X_train, X_test, y_train, y_test  = train_test_split(df_ma.iloc[:,:-1], df_ma["score_절폭"],test_size = 0.2)

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_train, y_train)

0.6250791854345239

In [20]:
lr.score(X_test, y_test)

0.49865953766292714

In [21]:
model = SVR(kernel="linear", gamma="auto")
model.fit(X_train, y_train)

print("학습 결과 : ", model.score(X_train, y_train))
print("테스트 결과 : ", model.score(X_test, y_test))

학습 결과 :  0.6165171376519478
테스트 결과 :  0.4833525823617576


In [22]:
ridge   =  Ridge(alpha = 1000)
ridge.fit(X_train, y_train)
ridge.score(X_train,y_train)

0.6030086100420007

In [23]:
ridge.score(X_test, y_test)

0.4901134936740519

In [24]:
lasso = Lasso(alpha = 0.00001)
lasso.fit(X_train, y_train)
lasso.score(X_train, y_train)

0.6250791851911257

In [25]:
lasso.score(X_test, y_test)

0.49865883301526803

In [26]:
lgbm = LGBMRegressor()
lgbm.fit(X_train, y_train)
lgbm.score(X_train, y_train)

0.7875644741360479

In [27]:
lgbm.score(X_test,  y_test)

0.5188918842620618

In [ ]:
lgbm = LGBMRegressor()
lgbm.fit(X_train, y_train)


In [29]:
y_test

35     79.05
67     84.25
85     75.55
10     79.95
72     79.40
23     81.10
119    71.85
22     75.55
47     76.75
94     74.85
92     77.50
79     79.15
50     81.10
89     78.55
18     79.95
28     78.35
48     74.85
16     71.20
14     75.85
45     78.15
80     72.70
3      73.15
65     76.80
86     80.30
95     73.15
Name: score_절폭, dtype: float64

In [30]:
lgbm.predict(X_test)

array([80.41298605, 82.32092404, 76.29447911, 77.56665564, 78.54068789,
       79.81590983, 75.87872035, 75.63477549, 78.63460114, 75.00476922,
       75.57424371, 77.73039992, 81.43860859, 77.05959834, 76.03777033,
       77.12682771, 77.1526262 , 72.7081263 , 74.97642326, 76.22128173,
       78.07191497, 76.8575412 , 78.16851016, 77.54133565, 74.0466749 ])

In [31]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [33]:
mean_absolute_error(lgbm.predict(X_test), y_test)

1.8296910245574367